In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', None) # remove for actual version
pd.set_option('display.max_columns', None) # remove for actual ver

In [2]:
data_string = "broockman_kalla_replication_data.dta"
df = pd.read_stata(data_string)
df

,id,vf_age,vf_party,vf_racename,vf_female,vf_black,vf_white,vf_hispanic,vf_vg_14,vf_vg_12,vf_vg_10,vf_democrat,vf_republican,miami_trans_law_t0,miami_trans_law2_t0,gender_norm_daugher_t0,gender_norm_looks_t0,gender_norm_rights_t0,gender_norms_sexchange_t0,gender_norms_moral_t0,gender_norms_abnormal_t0,gender_norm_trans_moral_wrong_t0,ssm_t0,therm_obama_t0,therm_gay_t0,therm_trans_t0,therm_marijuana_t0,therm_afams_t0,ideology_t0,religious_t0,respondent_t0,exposure_gay_t0,exposure_trans_t0,pid_t0,scale_for_blocking_t0,survey_language_t0,miami_trans_law_t1,miami_trans_law2_t1,therm_obama_t1,therm_trans_t1,therm_marijuana_t1,gender_norm_looks_t1,gender_norm_rights_t1,gender_norm_sexchange_t1,gender_norm_moral_t1,gender_norm_abnormal_t1,gender_norm_trans_moral_wrong_t1,respondent_t1,miami_trans_law_t2,miami_trans_law2_t2,therm_obama_t2,therm_jbush_t2,therm_trans_t2,therm_police_t2,therm_firefighters_t2,gender_norm_looks_t2,gender_norm_rights_t2,gender_norm_moral_t2,gender_norm_dress_t2,gender_norm_sexchange_t2,gender_norm_abnormal_t2,gender_norm_trans_moral_wrong_t2,trans_teacher_t2,trans_bathroom_t2,respondent_t2,miami_trans_law_withdef_t3,miami_trans_law2_withdef_t3,therm_obama_t3,therm_jbush_t3,therm_trans_t3,therm_police_t3,therm_firefighters_t3,gender_norm_looks_t3,gender_norm_rights_t3,gender_norm_moral_t3,gender_norm_dress_t3,gender_norm_sexchange_t3,gender_norm_abnormal_t3,gender_norm_trans_moral_wrong_t3,trans_teacher_t3,trans_bathroom_t3,trans_ad_displayed_t3,trans_law_post_ad_t3,respondent_t3,nfc_t3,miami_trans_law_withdef_t4,miami_trans_law2_withdef_t4,therm_obama_t4,therm_trans_t4,therm_mousavi_t4,gender_norm_looks_t4,gender_norm_rights_t4,gender_norm_moral_t4,gender_norm_dress_t4,gender_norm_sexchange_t4,gender_norm_abnormal_t4,gender_norm_trans_moral_wrong_t4,trans_teacher_t4,trans_bathroom_t4,respondent_t4,marijuana_pread1,marijuana_pread2,marijuana_postad,exp_actual_convo,contacted,survey_language_es,sdo_scale,treat_ind,canvass_recycling_rating,canvass_trans_ratingstart,canvass_trans_ratingsecond,canvass_trans_ratingend,canvass_minutes,canvasser_experience,canvasser_trans,canvasser_id,hh_id,block_ind,cluster_level_t0_scale_mean,vf_independent
0,60291,23.0,,Hispanic,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,NaN,NaN,,,NaN,NaN
1,52944,38.0,,Hispanic,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,NaN,NaN,,,NaN,NaN
2,27568,48.0,,African American,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,NaN,NaN,,,NaN,NaN
3,41146,NaN,,Hispanic,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [7]:
df[["respondent_t1"]]

,respondent_t1
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
68373,0.0
68374,1.0
68375,1.0
68376,0.0


In [ ]:
compute.factor.dv <- function(dv.names, respondent.booleans, print.loadings = TRUE){
  responders <- data[respondent.booleans,]
  
  # Factor analysis
  factor.obj <- princomp(responders[, dv.names], cor=TRUE)
  if(print.loadings) print(loadings(factor.obj))
  dv <- as.vector(factor.obj$scores[,1])
  
  # More positive values on the factor should indicate more tolerance; reverse otherwise.
  if(cor(dv, responders$miami_trans_law_t0, use="complete.obs") < 0) dv <- -1 * dv
  
  # Put in the order of the main data frame
  dv.in.order <- dv[match(data$id, responders$id)]
  
  # Rescale to mean 0 sd 1 in placebo group; treatment effects can then be interpreted
  # as the effect in standard deviations the treatment would have among an untreated
  # population.
  dv.in.order <- (dv.in.order - mean(dv.in.order[!data$treat_ind], na.rm=TRUE)) /
    sd(dv.in.order[!data$treat_ind], na.rm=TRUE)
  
  return(as.vector(dv.in.order))
}

compute.factor.dv(all.dv.names.t1, data$respondent_t1==1 & !is.na(data$respondent_t1))

In [19]:
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer

In [49]:
df2=df[df["respondent_t1"]==1][['miami_trans_law_t1', 'miami_trans_law2_t1', 'therm_trans_t1',
                      'gender_norm_sexchange_t1', 'gender_norm_moral_t1',
                      'gender_norm_abnormal_t1', 'gender_norm_trans_moral_wrong_t1']]

# Issue Here:
* Values do not match up despite trying 3 different methods

In [81]:
pca = PCA(n_components=7)

X = pca.fit_transform(scale(df2.values))

loadings = pd.DataFrame(np.round(pca.components_.T, decimals=3) ,
                        columns=['PC1', 'PC2', "PC3", "PC4", "PC5", "PC6", "PC7"],
                        index=df2.columns)

loadings * -1


,PC1,PC2,PC3,PC4,PC5,PC6,PC7
miami_trans_law_t1,0.366,0.533,0.217,0.354,0.095,-0.159,-0.612
miami_trans_law2_t1,0.385,0.501,0.171,0.077,0.017,0.126,0.741
therm_trans_t1,0.395,0.108,-0.003,-0.894,0.012,-0.036,-0.178
gender_norm_sexchange_t1,0.329,0.057,-0.923,0.156,-0.112,0.022,-0.013
gender_norm_moral_t1,-0.388,0.434,-0.078,-0.116,-0.272,0.731,-0.180
gender_norm_abnormal_t1,-0.388,0.326,-0.241,-0.126,0.805,-0.132,0.053
gender_norm_trans_moral_wrong_t1,-0.391,0.393,-0.092,-0.125,-0.506,-0.636,0.091


In [40]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

loading_matrix = pd.DataFrame(loadings, columns=['PC1', 'PC2', "PC3", "PC4", "PC5", "PC6", "PC7"],
                        index=df2.columns)
loading_matrix

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
miami_trans_law_t1,-1.006144,-1.153008,-0.566631,0.110616,-0.419743,-0.138274,-0.086840
miami_trans_law2_t1,-1.111721,-0.987981,-0.492076,0.049049,0.462517,0.207830,0.130869
therm_trans_t1,-28.393778,0.131249,-0.018801,0.003136,-0.004217,-0.001322,-0.001092
gender_norm_sexchange_t1,-0.653758,-0.452814,0.226760,-0.914675,0.010495,-0.069385,-0.021494
gender_norm_moral_t1,0.756328,0.503941,-0.730260,-0.112753,0.181205,-0.080940,-0.448686
gender_norm_abnormal_t1,0.739979,0.526479,-0.577428,-0.229395,-0.254565,0.485716,0.115562
gender_norm_trans_moral_wrong_t1,0.761345,0.527301,-0.687145,-0.115677,0.039832,-0.379811,0.350558


In [12]:

transformer = FactorAnalysis(n_components=7, random_state=0)
X_transformed = transformer.fit_transform(df2)
X_transformed

array([[ 1.91663657,  0.20319241, -0.78892535, ...,  0.        ,
         0.        ,  0.        ],
       [-1.39116237,  0.76960529, -0.92381429, ...,  0.        ,
         0.        ,  0.        ],
       [-1.49895974,  0.50174685, -0.49933508, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.25729114, -0.1269192 , -0.78766038, ...,  0.        ,
         0.        ,  0.        ],
       [-0.94222142, -0.36081931, -0.08646259, ...,  0.        ,
         0.        ,  0.        ],
       [-0.27461515, -0.60469011, -0.08129917, ...,  0.        ,
         0.        ,  0.        ]])

In [47]:
from sklearn.preprocessing import MinMaxScaler

In [78]:
fa = FactorAnalyzer(n_factors=6,rotation='promax', is_corr_matrix=True)
fa.fit(df2)
print(pd.DataFrame(fa.loadings_,index=df2.columns))

LinAlgError: Last 2 dimensions of the array must be square

In [50]:
fa = FactorAnalyzer(n_factors=7, rotation="varimax")
scaler = MinMaxScaler()
testDF = scaler.fit_transform(df2)
fa.fit(testDF)
loading_matrix = pd.DataFrame(fa.loadings_, columns=['PC1', 'PC2', "PC3", "PC4", "PC5", "PC6", "PC7"],
                        index=df2.columns)
loading_matrix

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
miami_trans_law_t1,-0.259457,0.833723,0.173832,-0.050293,-0.045066,-0.038124,0.0
miami_trans_law2_t1,-0.262967,0.866335,0.240747,0.099820,0.059739,0.038410,0.0
therm_trans_t1,-0.453849,0.483161,0.463479,0.192768,0.008960,0.021793,0.0
gender_norm_sexchange_t1,-0.365396,0.339304,0.442450,-0.037830,0.001811,-0.010074,0.0
gender_norm_moral_t1,0.825498,-0.224434,-0.251187,0.033247,0.085478,-0.047329,0.0
gender_norm_abnormal_t1,0.748019,-0.322780,-0.157456,-0.157810,0.035759,0.071254,0.0
gender_norm_trans_moral_wrong_t1,0.796799,-0.259985,-0.245706,0.027677,-0.142589,-0.014679,0.0


In [1]:
def compute_factor_dv(dv_names, respondent_booleans, print_loadings = True):
    responders = 

SyntaxError: unexpected EOF while parsing (<ipython-input-1-c84e05095e59>, line 1)

In [9]:
# bysort hh_id: egen scale_t0_hh_avg = mean(scale_for_blocking_t0)

df[["hh_id","id","scale_for_blocking_t0"]]

,hh_id,id,scale_for_blocking_t0
0,,60291,NaN
1,,52944,NaN
2,,27568,NaN
3,,41146,NaN
4,,17510,NaN
...,...,...,...
68373,june6_9943,68102,1.515273
68374,june6_9971,42009,0.626062
68375,june6_9971,42061,0.369057
68376,june6_9976,25963,0.620112


In [ ]:
### Estimate the treatment effect for ALL canvassers
t1.all <- est.ate(data$trans.tolerance.dv.t1)
t2.all <- est.ate(data$trans.tolerance.dv.t2)
t3.all <- est.ate(data$trans.tolerance.dv.t3)
t4.all <- est.ate(data$trans.tolerance.dv.t4)

### Estimate the treatment effect by trans and cis canvassers
###At 3 day survey
t1.trans <- est.ate(data$trans.tolerance.dv.t1,
                    data$canvasser_trans == 1)
t1.cis <- est.ate(data$trans.tolerance.dv.t1,
                  data$canvasser_trans == 0)

###At 3 week survey
t2.trans <- est.ate(data$trans.tolerance.dv.t2,
                    data$canvasser_trans == 1)
t2.cis <- est.ate(data$trans.tolerance.dv.t2,
                  data$canvasser_trans == 0)

###At 6 week survey
t3.trans <- est.ate(data$trans.tolerance.dv.t3,
                    data$canvasser_trans == 1)
t3.cis <- est.ate(data$trans.tolerance.dv.t3,
                  data$canvasser_trans == 0)

###At 3 month survey
t4.trans <- est.ate(data$trans.tolerance.dv.t4,
                    data$canvasser_trans == 1)
t4.cis <- est.ate(data$trans.tolerance.dv.t4,
                  data$canvasser_trans == 0)


### Make DF of summary stats
summary.stats.df <- as.data.frame(rbind(t1.all, t2.all,
                          t3.all, t4.all,
                          t1.trans, t1.cis,
                          t2.trans, t2.cis,
                          t3.trans, t3.cis,
                          t4.trans, t4.cis),
                          stringsAsFactors = FALSE)

### Change from strings back to numeric, and remove t- and p-values, which are not used.
summary.stats.df <- summary.stats.df[,1:2]
summary.stats.df[,1] <- as.numeric(summary.stats.df[,1])
summary.stats.df[,2] <- as.numeric(summary.stats.df[,2])

### Better variable names
names(summary.stats.df) <- c("point.estimate", "se")

### Map row names of summary.stats.df into days
unique.days <- c(3, 3*7, 6*7, 12*7)
summary.stats.df$days <- unique.days[as.numeric(substr(row.names(summary.stats.df), 2, 2))]

### Read canvasser group from row names
canvasser.label.map <- list(all = "All",
                            tra = "Transgender/Gender\nNon-Conforming Only",
                            cis = "Non-Transgender Only")
summary.stats.df$Canvasser <- factor(as.character(
  canvasser.label.map[substr(row.names(summary.stats.df), 4, 6)]
  ))

### X position of different canvasser groups
summary.stats.df$xpos <- with(summary.stats.df, days + as.numeric(Canvasser) -
                                mean(as.numeric(Canvasser)))

### Point estimate Y
summary.stats.df$point.estimate.y <- summary.stats.df$point.estimate
  ### Fix text overlap of point estimate labels.
  summary.stats.df$point.estimate.y[9] <- summary.stats.df$point.estimate.y[9] + .003
  summary.stats.df$point.estimate.y[3] <- summary.stats.df$point.estimate.y[3] - .003

### Compute CIs
summary.stats.df$se.high <- summary.stats.df$point.estimate + summary.stats.df$se
summary.stats.df$se.low <- summary.stats.df$point.estimate - summary.stats.df$se
summary.stats.df$ci.high <- summary.stats.df$point.estimate + summary.stats.df$se * 1.96
summary.stats.df$ci.low <- summary.stats.df$point.estimate - summary.stats.df$se * 1.96

summary.stats.df$point.estimate.label <- paste0(round(summary.stats.df$point.estimate,
                                                      2)," SDs")

g <- ggplot(summary.stats.df,
            aes(x=xpos, y=point.estimate,
                group=Canvasser, color=Canvasser)) +
  theme_classic() +
  ### CIs
  geom_linerange(aes(ymin=se.low, ymax=se.high), lwd=1) +
  geom_linerange(aes(ymin=ci.low, ymax=ci.high)) +
  ### Point estimate points
  geom_point(color="black") +
  ### Point estimate markers
  annotate("text", label=summary.stats.df$point.estimate.label,
           x = summary.stats.df$xpos + 6.3,
           y = summary.stats.df$point.estimate.y,
           size = 3) +
  ### Canvassing treatment line / label
  geom_vline(xintercept = 0, linetype = "dashed") +
  annotate("text", label = "Canvassing Treatment",
           x = -1.5, y = .3, size = 3.5, angle = 90) +
  ### Day labels
  annotate("text",
           label = c("+3 Days", "+3 Weeks", "+6 Weeks", "+3 Months"),
           x = unique.days, y = -.04,
           colour = "black", size = 3) +
  ### Y axis
  ylab("Effect on Transgender Tolerance Scale, in Standard Deviations") + 
  scale_y_continuous() +
  ### X axis
  xlab("Days After Canvassing Treatment") +
  geom_hline(yintercept = 0) +
  ### Overall Title and Legend
  ggtitle("Differences Between Treatment and Placebo") +
  guides(fill=FALSE) +
  theme(legend.position = "bottom")

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import scale

# load iris dataset
iris = datasets.load_iris()

X = scale(iris.data)
y = iris.target

# apply PCA
pca = decomposition.PCA(n_components=2)
X = pca.fit_transform(X)


In [42]:
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=iris.feature_names)
loadings

,PC1,PC2
sepal length (cm),0.521066,0.377418
sepal width (cm),-0.269347,0.923296
petal length (cm),0.580413,0.024492
petal width (cm),0.564857,0.066942


In [45]:
iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [ ]:
est.ate <- function(dv, include.obs = NULL, include.covariates = TRUE){
  if(is.null(include.obs)){
    include.obs <- !is.na(dv) 
  }
  include.obs <- which(include.obs & !is.na(dv))
  
  if(include.covariates) lm.obj <- lm(dv[include.obs] ~ data$treat_ind[include.obs] +
                                        x[include.obs,])
  if(!include.covariates) lm.obj <- lm(dv[include.obs] ~ data$treat_ind[include.obs])
  
  # Calculate cluster-robust standard errors.
  result <- cl(lm.obj, data$hh_id[include.obs])[2,]
  
  # Adjust point estimate and standard error for contact rate in subsample.
  itt_d <- lm(treatment.delivered ~ treat_ind, data[include.obs,])$coefficients[2]
  result[1:2] <- result[1:2] / itt_d
  
  # Per pre-analysis plan, rejection region is top 4% and bottom 1% of sampling distribution,
  # so p-values are reported for this region; otherwise, we write "n.s."
  # See Olken (*43*) , page 70, footnote 5.
  # Note that cl() returns two-tailed p-values that must be converted to one-tailed.
  # Note that all DVs were recoded such that higher values indicated more tolerance.
  result[4] <- result[4] / 2 # p-value corresponds to mass under one side of distribution.
  rejection.region <- ((result[4] < .04 & result[1] > 0) | # Significant positive result.
                         (result[4] < .01 & result[1] < 0)) # Significant negative result.
  result <- round(result, 3)
  if(rejection.region) result[4] <- paste0(as.character(result[4]), "*")
  if(!rejection.region) result[4] <- "*n.s.*"
  if(result[4] == "0*") result[4] <- "0.000*" # Indicate precision of 0 p-value.
  names(result)[4] <- "*p*"
  return(result)
}

In [93]:
df = pd.DataFrame({"col1":[0,1,np.nan,3],"col2":[2,3,4,np.nan]})
~df.isna()

,col1,col2
0,True,True
1,True,True
2,False,True
3,True,False


In [ ]:
def est_ate(df, include_obs = "null", include_covariates = True):
    if (include == "null"):
        df = ~df.isna()
    df = df[~df.isna()]